In [1]:
from sporgboost.forests import *
from sklearn.metrics import roc_auc_score
from get_data import load
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sporgboost.preprocessing import onehot_encode
X, y = load("data/sparse_parity.parquet")
# X, y = load_iris(return_X_y=True)
# y = onehot_encode(y)


In [2]:
from sporgboost.trees import RotationalDecisionTree
rot = RotationalDecisionTree(K=2)
rot.fit(X, y)

In [3]:
from sporgboost.forests import *
rf = RandomForest()
rf.fit(X, y)

In [9]:
sporf = SPORF(d=2, s=3)
sporf.fit(X, y)

In [5]:
rrf = RotationalForest(K=2)
rrf.fit(X, y)

In [19]:
# models = {
#     # 'rf' : RandomForest(),
#     # 'ab' : AdaBoost(),
#     'sporf' : SPORF(d=2, s=3),
#     'sporgboost' : SPORGBoost(d=2, s=3, max_depth=2),
#     # 'rrf' : RotationalRandomForest(K=2),
#     # 'rotboost' : RotBoost(K=2)
# }

# for k, m in models.items():
#     print(f"Fitting {k}")
#     m.fit(X_train, y_train)


Fitting sporf
Fitting sporgboost


In [6]:
roc_auc_score(y, rrf.predict(X), multi_class='ovo')

0.6357403228749623

In [10]:
roc_auc_score(y, sporf.predict_proba(X), multi_class='ovo')

0.976297372398488

In [8]:
roc_auc_score(y, rf.predict_proba(X), multi_class='ovo')

0.5819648490844376

In [24]:
from sklearn.metrics import accuracy_score
# accuracy_score(y_test, rf.predict(X_test))

In [26]:
# Get AUC scores
auc = {key : accuracy_score(y_test, model.predict(X_test)) for key, model in models.items() if key in {'sporf','sporgboost'}}
auc

{'sporf': 0.7630681818181818, 'sporgboost': 0.6098863636363636}